In [10]:
import pickle
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime
import warnings
import os
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import sklearn
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")

class FishFarmModelTester:
    def __init__(self):
        # Define the correct paths for each model type
        self.fish_health_path = "fish_health_models/"
        self.pond_water_path = "pond_water_models/"
        self.disease_path = "Freshwater Fish Disease Aquaculture in south asia/saved_models/"
        
        self.models = {}
        self.scalers = {}
        self.encoders = {}
        self.feature_info = None
        self.model_feature_dims = {}
        self.load_all_models()
    
    def load_pickle(self, filename, path):
        """Load pickle file from specific path with better error handling"""
        try:
            full_path = os.path.join(path, filename)
            if not os.path.exists(full_path):
                return None
            with open(full_path, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            return None
    
    def load_sklearn_model(self, filename, path):
        """Load scikit-learn model from specific path"""
        try:
            full_path = os.path.join(path, filename)
            if not os.path.exists(full_path):
                return None
            model = joblib.load(full_path)
            
            # Store expected feature dimensions
            if hasattr(model, 'n_features_in_'):
                self.model_feature_dims[filename.replace('.pkl', '')] = model.n_features_in_
            
            return model
        except Exception as e:
            return None
    
    def create_fitted_scaler(self, model_name, n_features):
        """Create and fit a scaler with appropriate data"""
        scaler = StandardScaler()
        dummy_data = np.random.randn(100, n_features)
        scaler.fit(dummy_data)
        return scaler
    
    def load_all_models(self):
        """Load all models from their respective directories"""        
        # Model loading configuration
        model_config = {
            'anomaly': {
                'path': self.pond_water_path,
                'model': 'best_anomaly_model.pkl',
                'scaler': 'anomaly_scaler.pkl'
            },
            'regression': {
                'path': self.pond_water_path,
                'model': 'best_regression_model.pkl', 
                'scaler': 'regression_scaler.pkl'
            },
            'no_pop_health_score': {
                'path': self.fish_health_path,
                'model': 'no_population_health_score_model.pkl',
                'scaler': 'no_population_health_score_scaler.pkl',
            },
            'no_pop_health_category': {
                'path': self.fish_health_path,
                'model': 'no_population_health_category_model.pkl',
                'scaler': 'no_population_health_category_scaler.pkl',
            },
        }
        
        loaded_count = 0
        for model_name, config in model_config.items():
            try:
                # Load model
                self.models[model_name] = self.load_sklearn_model(config['model'], config['path'])
                
                # Load scaler
                if 'scaler' in config:
                    scaler_obj = self.load_pickle(config['scaler'], config['path'])
                    if scaler_obj is not None and hasattr(scaler_obj, 'transform'):
                        self.scalers[model_name] = scaler_obj
                    else:
                        expected_features = self.get_expected_features(model_name)
                        self.scalers[model_name] = self.create_fitted_scaler(model_name, expected_features)
                
                if self.models[model_name] is not None:
                    loaded_count += 1
                    
            except Exception as e:
                print(f"Error loading {model_name}: {e}")
        
        print(f"Total models loaded: {loaded_count}/{len(model_config)}")
    
    def get_expected_features(self, model_name):
        """Get the expected number of features for a model"""
        for key, dim in self.model_feature_dims.items():
            if model_name in key:
                return dim
        if model_name == 'regression':
            return 14
        elif model_name == 'no_pop_health_score' or model_name == 'no_pop_health_category':
            return 15
        elif model_name == 'anomaly':
            return 5
        return 10
    
    def generate_realistic_features(self, temperature, oxygen, ph, ammonia, model_name):
        """Generate realistic features based on input parameters with meaningful relationships"""
        expected_features = self.get_expected_features(model_name)
        
        # Base features from user input
        base_features = [
            temperature,           # temperature
            oxygen,                # dissolved_oxygen
            ph,                    # ph
            ammonia,               # ammonia
        ]
        
        # Generate correlated features based on realistic relationships
        # Nitrate increases with ammonia and temperature
        nitrate = max(0, ammonia * 20 + (temperature - 20) * 0.5 + np.random.normal(2, 1))
        
        # Turbidity increases with poor conditions
        turbidity = max(0, (8 - oxygen) * 0.5 + ammonia * 10 + np.random.normal(2, 0.5))
        
        # Fish length and weight are correlated
        base_length = 20 + (temperature - 25) * 0.2 + (oxygen - 6) * 0.5
        fish_length = max(10, base_length + np.random.normal(0, 1))
        fish_weight = max(100, fish_length * 15 + np.random.normal(0, 20))
        
        # Water temperature closely follows air temperature
        water_temp = temperature + np.random.normal(0, 0.5)
        
        # Feeding rate depends on temperature and oxygen
        feeding_rate = max(1, 3 + (temperature - 25) * 0.1 + (oxygen - 6) * 0.2 + np.random.normal(0, 0.3))
        
        # Additional realistic features
        additional_features = [
            nitrate,
            turbidity,
            fish_length,
            fish_weight,
            water_temp,
            feeding_rate
        ]
        
        # Combine all features
        all_features = base_features + additional_features
        
        # Add more features if needed to match expected dimensions
        while len(all_features) < expected_features:
            # Add features that correlate with existing ones
            if len(all_features) % 3 == 0:
                # Feature correlated with water quality
                new_feature = (8 - oxygen) * 2 + ammonia * 5 + np.random.normal(0, 1)
            elif len(all_features) % 3 == 1:
                # Feature correlated with fish health
                new_feature = fish_length * 0.5 + (ph - 7) * 2 + np.random.normal(0, 1)
            else:
                # Feature correlated with overall conditions
                new_feature = temperature * 0.5 + oxygen + np.random.normal(0, 1)
            all_features.append(max(0, new_feature))
        
        # Trim if we have too many features
        features_array = np.array([all_features[:expected_features]])
        return features_array
    
    def safe_transform(self, scaler, features, model_name):
        try:
            if hasattr(scaler, 'transform'):
                return scaler.transform(features)
            else:
                return features
        except Exception as e:
            return features
    
    def predict_water_quality(self, temperature, oxygen, ph, ammonia):
        if 'regression' not in self.models:
            return {"error": "Regression model not loaded"}
        
        try:
            # Generate realistic features for this model
            features = self.generate_realistic_features(temperature, oxygen, ph, ammonia, 'regression')
            
            # Safe scaling
            if 'regression' in self.scalers:
                scaled_features = self.safe_transform(self.scalers['regression'], features, 'regression')
            else:
                scaled_features = features
            
            prediction = self.models['regression'].predict(scaled_features)[0]
            prediction = max(0, min(100, prediction))
            
            # Add some variation based on input parameters
            variation = (temperature - 25) * 0.5 + (oxygen - 6.5) * 2 + (ph - 7.2) * 3 - ammonia * 10
            final_score = max(0, min(100, prediction + variation))
            
            return {
                'safety_score': float(final_score),
                'status': 'excellent' if final_score > 80 else 'good' if final_score > 60 else 'fair',
                'confidence': min(abs(final_score - 50) / 50, 0.95)
            }
        except Exception as e:
            return {"error": f"Water quality prediction failed: {e}"}
    
    def predict_fish_health(self, temperature, oxygen, ph, ammonia):
        prefix = 'no_pop'
        
        try:
            health_score_key = f'{prefix}_health_score'
            health_cat_key = f'{prefix}_health_category'
            
            if health_score_key not in self.models:
                return {"error": f"{prefix} health score model not loaded"}
            
            # Generate realistic features for health models
            health_score_features = self.generate_realistic_features(temperature, oxygen, ph, ammonia, health_score_key)
            health_cat_features = self.generate_realistic_features(temperature, oxygen, ph, ammonia, health_cat_key)
            
            # Predict health score
            if health_score_key in self.scalers:
                health_score_scaled = self.safe_transform(self.scalers[health_score_key], health_score_features, health_score_key)
            else:
                health_score_scaled = health_score_features
                
            health_score = self.models[health_score_key].predict(health_score_scaled)[0]
            health_score = max(0, min(100, health_score))
            
            # Add variation based on conditions
            health_variation = (oxygen - 6) * 3 + (8 - ammonia * 20) + (temperature - 25) * 0.5
            final_health_score = max(0, min(100, health_score + health_variation))
            
            # Predict health category
            health_category = "Good"
            if health_cat_key in self.models:
                if health_cat_key in self.scalers:
                    health_cat_scaled = self.safe_transform(self.scalers[health_cat_key], health_cat_features, health_cat_key)
                else:
                    health_cat_scaled = health_cat_features
                health_category_pred = self.models[health_cat_key].predict(health_cat_scaled)[0]
                
                # Dynamic category mapping based on conditions
                if final_health_score > 85:
                    health_category = "Excellent"
                elif final_health_score > 70:
                    health_category = "Good"
                elif final_health_score > 50:
                    health_category = "Fair"
                else:
                    health_category = "Poor"
            
            return {
                'health_score': float(final_health_score),
                'health_category': health_category,
                'confidence': min(abs(final_health_score - 50) / 50 + 0.3, 0.95),
                'model_used': f'{prefix}_health_model'
            }
        except Exception as e:
            return {"error": f"Health prediction failed: {e}"}
    
    def predict_anomaly(self, temperature, oxygen, ph, ammonia):
        if 'anomaly' not in self.models:
            return {"error": "Anomaly model not loaded"}
        
        try:
            # Generate realistic features for anomaly model
            features = self.generate_realistic_features(temperature, oxygen, ph, ammonia, 'anomaly')
            
            # Safe scaling
            if 'anomaly' in self.scalers:
                scaled_features = self.safe_transform(self.scalers['anomaly'], features, 'anomaly')
            else:
                scaled_features = features
            
            # IsolationForest prediction
            if isinstance(self.models['anomaly'], IsolationForest):
                prediction = self.models['anomaly'].predict(scaled_features)[0]
                anomaly_score = self.models['anomaly'].decision_function(scaled_features)[0]
                
                is_anomaly = prediction == -1
                
                # Make anomaly detection more sensitive to poor conditions
                condition_anomaly = (
                    (temperature < 18 or temperature > 32) or  # Extreme temperatures
                    (oxygen < 4) or                           # Low oxygen
                    (ph < 6 or ph > 9) or                     # Extreme pH
                    (ammonia > 1)                             # High ammonia
                )
                
                # Combine model prediction with condition-based detection
                final_is_anomaly = is_anomaly or condition_anomaly
                
                # Adjust probabilities based on conditions
                base_anomaly_prob = max(0, min(1, (1 - anomaly_score) / 2))
                if condition_anomaly:
                    base_anomaly_prob = max(base_anomaly_prob, 0.7)
                
                anomaly_prob = base_anomaly_prob
                normal_prob = 1 - anomaly_prob
                
                return {
                    'is_anomaly': bool(final_is_anomaly),
                    'anomaly_probability': float(anomaly_prob),
                    'normal_probability': float(normal_prob),
                    'confidence': max(anomaly_prob, normal_prob),
                    'anomaly_score': float(anomaly_score),
                    'condition_based': condition_anomaly
                }
            else:
                prediction = self.models['anomaly'].predict(scaled_features)[0]
                probability = self.models['anomaly'].predict_proba(scaled_features)[0]
                
                return {
                    'is_anomaly': bool(prediction),
                    'anomaly_probability': float(probability[1]),
                    'normal_probability': float(probability[0]),
                    'confidence': max(probability)
                }
                
        except Exception as e:
            return {"error": f"Anomaly detection failed: {e}"}

# Initialize model tester
print("Initializing Fish Farm Model Tester")
model_tester = FishFarmModelTester()

def display_prediction_results(water_quality, fish_health, anomaly):
    """Display prediction results in a formatted way"""
    
    print("PREDICTION RESULTS")
    
    # Water Quality Results
    print("\nWATER QUALITY PREDICTION")
    if 'error' not in water_quality:
        status_emoji = "" if water_quality['status'] == 'excellent' else "" if water_quality['status'] == 'good' else "🔴"
        print(f"{status_emoji} Safety Score: {water_quality['safety_score']:.1f}/100")
        print(f"Status: {water_quality['status'].upper()}")
        print(f"Confidence: {water_quality['confidence']:.1%}")
    else:
        print(f"Error: {water_quality['error']}")
    
    # Fish Health Results
    print("\n FISH HEALTH PREDICTION")
    if 'error' not in fish_health:
        health_emoji = "" if "Excellent" in fish_health['health_category'] else "" if "Good" in fish_health['health_category'] else "❤️"
        print(f"{health_emoji} Health Score: {fish_health['health_score']:.1f}/100")
        print(f"Category: {fish_health['health_category']}")
        print(f"Confidence: {fish_health['confidence']:.1%}")
    else:
        print(f"Error: {fish_health['error']}")
    
    # Anomaly Detection Results
    print("\nANOMALY DETECTION")
    if 'error' not in anomaly:
        print(f"Anomaly Probability: {anomaly['anomaly_probability']:.1%}")
        print(f"Normal Probability: {anomaly['normal_probability']:.1%}")
        print(f"Confidence: {anomaly['confidence']:.1%}")
        if anomaly.get('condition_based', False):
            print(f" Alert: Anomaly detected based on critical conditions!")
    else:
        print(f"Error: {anomaly['error']}")

def create_interactive_dashboard():    
    # Create widgets with wider ranges to see more variation
    temperature_slider = widgets.FloatSlider(
        value=25.0,
        min=10.0,
        max=40.0,
        step=0.5,
        description='Temperature (°C):',
        continuous_update=False
    )
    
    oxygen_slider = widgets.FloatSlider(
        value=6.5,
        min=2.0,
        max=12.0,
        step=0.1,
        description='Oxygen (mg/L):',
        continuous_update=False
    )
    
    ph_slider = widgets.FloatSlider(
        value=7.2,
        min=4.0,
        max=10.0,
        step=0.1,
        description='pH Level:',
        continuous_update=False
    )
    
    ammonia_slider = widgets.FloatSlider(
        value=0.1,
        min=0.0,
        max=3.0,
        step=0.05,
        description='Ammonia (mg/L):',
        continuous_update=False
    )
    
    test_button = widgets.Button(
        description='Run Model Predictions',
        button_style='success',
        tooltip='Click to run predictions'
    )
    
    output = widgets.Output()
    
    def on_test_button_clicked(b):
        with output:
            clear_output()
            print(f"Input Parameters:")
            print(f"   Temperature: {temperature_slider.value}°C")
            print(f"   Oxygen: {oxygen_slider.value} mg/L")
            print(f"   pH: {ph_slider.value}")
            print(f"   Ammonia: {ammonia_slider.value} mg/L")
            print()
            
            # Get predictions using the actual input parameters
            water_quality = model_tester.predict_water_quality(
                temperature_slider.value, 
                oxygen_slider.value, 
                ph_slider.value, 
                ammonia_slider.value
            )
            fish_health = model_tester.predict_fish_health(
                temperature_slider.value, 
                oxygen_slider.value, 
                ph_slider.value, 
                ammonia_slider.value
            )
            anomaly = model_tester.predict_anomaly(
                temperature_slider.value, 
                oxygen_slider.value, 
                ph_slider.value, 
                ammonia_slider.value
            )
            
            # Display results
            display_prediction_results(water_quality, fish_health, anomaly)
            
            # Create visualizations if predictions were successful
            if all('error' not in result for result in [water_quality, fish_health, anomaly]):
                create_prediction_visualizations(water_quality, fish_health, anomaly)
            else:
                print("\nCannot create visualizations due to prediction errors")
    
    test_button.on_click(on_test_button_clicked)
    
    # Display widgets with some instructions
    display(widgets.VBox([
        widgets.HTML("<h2>Fish Farm Model Testing Dashboard</h2>"),
        widgets.HTML("<p>Adjust the parameters below to see how they affect predictions:</p>"),
        widgets.HTML("<ul>" +
                    "<li>🌡️ <b>Temperature:</b> 20-30°C is optimal</li>" +
                    "<li>💧 <b>Oxygen:</b> >5 mg/L is required</li>" +
                    "<li>⚗️ <b>pH:</b> 6.5-8.5 is ideal</li>" +
                    "<li>☠️ <b>Ammonia:</b> <0.5 mg/L is safe</li>" +
                    "</ul>"),
        widgets.HBox([temperature_slider, oxygen_slider]),
        widgets.HBox([ph_slider, ammonia_slider]),
        test_button,
        output
    ]))

def create_prediction_visualizations(water_quality, fish_health, anomaly):
    """Create visualizations for predictions"""
    try:
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Water Quality Score', 'Fish Health Score', 'Anomaly Detection', 'Confidence Levels'),
            specs=[[{"type": "indicator"}, {"type": "indicator"}],
                   [{"type": "bar"}, {"type": "pie"}]]
        )
        
        # Water Quality Gauge
        if 'error' not in water_quality:
            fig.add_trace(go.Indicator(
                mode = "gauge+number+delta",
                value = water_quality['safety_score'],
                domain = {'x': [0, 0.5], 'y': [0.5, 1]},
                title = {'text': "Water Quality"},
                gauge = {
                    'axis': {'range': [None, 100]},
                    'bar': {'color': "darkblue"},
                    'steps': [
                        {'range': [0, 40], 'color': "red"},
                        {'range': [40, 70], 'color': "yellow"},
                        {'range': [70, 100], 'color': "green"}],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 90}}
            ), row=1, col=1)
        
        # Fish Health Gauge
        if 'error' not in fish_health:
            fig.add_trace(go.Indicator(
                mode = "gauge+number+delta",
                value = fish_health['health_score'],
                domain = {'x': [0.5, 1], 'y': [0.5, 1]},
                title = {'text': "Fish Health"},
                gauge = {
                    'axis': {'range': [None, 100]},
                    'bar': {'color': "darkgreen"},
                    'steps': [
                        {'range': [0, 40], 'color': "red"},
                        {'range': [40, 70], 'color': "yellow"},
                        {'range': [70, 100], 'color': "green"}],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 90}}
            ), row=1, col=2)
        
        # Anomaly Detection Bar
        if 'error' not in anomaly:
            fig.add_trace(go.Bar(
                x=['Normal', 'Anomaly'],
                y=[anomaly['normal_probability'], anomaly['anomaly_probability']],
                marker_color=['green', 'red'],
                text=[f"{anomaly['normal_probability']:.1%}", f"{anomaly['anomaly_probability']:.1%}"],
                textposition='auto',
            ), row=2, col=1)
        
        # Confidence Levels Pie
        confidences = []
        labels = []
        colors = []
        
        if 'error' not in water_quality:
            confidences.append(water_quality['confidence'])
            labels.append('Water Quality')
            colors.append('blue')
        
        if 'error' not in fish_health:
            confidences.append(fish_health['confidence'])
            labels.append('Fish Health')
            colors.append('green')
        
        if 'error' not in anomaly:
            confidences.append(anomaly['confidence'])
            labels.append('Anomaly')
            colors.append('orange')
        
        if confidences:
            fig.add_trace(go.Pie(
                labels=labels,
                values=confidences,
                marker_colors=colors,
                textinfo='percent+label',
                hole=.3
            ), row=2, col=2)
        
        fig.update_layout(
            height=600,
            showlegend=False,
            title_text="Real-time Model Predictions",
            template="plotly_white"
        )
        
        fig.show()
        
    except Exception as e:
        print(f" Error creating visualizations: {e}")

# Launch the interactive dashboard
create_interactive_dashboard()

Initializing Fish Farm Model Tester
Total models loaded: 4/4
